In [1]:
import mysql.connector
import pandas as pd
from IPython.display import Markdown, display

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="secret",
  port="3306",
  database="serlo"
)

def printmd(string):
    display(Markdown(string))

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def query_number(sql):
    return query(sql)[0][0]

In [2]:
year = 2023

printmd(f"# Auswertung für das Jahr {year}")

# Auswertung für das Jahr 2023

In [3]:
count = query_number("select count(*) from event_log where YEAR(event_log.date) = %s and event_log.event_id IN (5);" % year)

printmd(f"#### Anzahl Bearbeitungen: {count}")

#### Anzahl Bearbeitungen: 7445

In [4]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "article" and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "article" and year(entity.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Artikel in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Artikel: {count}")

#### Anzahl neuer Artikel in diesem Jahr: 253

#### Aktueller Bestand von Artikel: 3815

In [5]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "course-page" and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name = "course-page"  and year(entity.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Kursseiten in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Kursen: {count}")

#### Anzahl neuer Kursseiten in diesem Jahr: 531

#### Aktueller Bestand von Kursen: 4223

In [6]:
new_count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name in ("text-exercise", "text-exercise-group") and year(entity.date) = %s
""" % year)
count = query_number("""
    select count(*) from entity
    join type on entity.type_id = type.id
    join uuid on entity.id = uuid.id
    where type.name in ("text-exercise", "text-exercise-group") and year(entity.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Aufgaben in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Aufgaben: {count}")

#### Anzahl neuer Aufgaben in diesem Jahr: 1258

#### Aktueller Bestand von Aufgaben: 8214

In [7]:
new_count = query_number("""
    select count(*) from comment
    join uuid on comment.id = uuid.id
    where uuid.trashed = 0 and year(comment.date) = %s
""" % year)
count = query_number("""
    select count(*) from comment
    join uuid on comment.id = uuid.id
    where uuid.trashed = 0 and year(comment.date) <= %s
""" % year)

printmd(f"#### Anzahl neuer Kommentare in diesem Jahr: {new_count}")
printmd(f"#### Aktueller Bestand von Kommentaren: {count}")

#### Anzahl neuer Kommentare in diesem Jahr: 1048

#### Aktueller Bestand von Kommentaren: 9253

In [8]:
printmd("### Liste von Autor:innen der Anzahl ihrer Bearbeitung Bearbeitungen")
pd.set_option('display.max_rows', None)
df = pd.read_sql("""
    select user.username, count(event_log.actor_id) as edits from event_log
    join user on user.id = event_log.actor_id
    where year(event_log.date) = %s
    group by event_log.actor_id
    order by edits desc
""" % year, db)
df.to_csv("/tmp/authors.csv")
df

### Liste von Autor:innen der Anzahl ihrer Bearbeitung Bearbeitungen

/tmp/ipykernel_84258/3915536109.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""


,username,edits
0,Felix_Eccardt,3652
1,Kowalsky,2611
2,Karin,2280
3,Sascha_Lill_95,2100
4,Corinna,1088
5,Peter,1085
6,metzgaria,812
7,KlausB,698
8,Alexandra-Wolfram,669
9,LinaMaria,589
